Author: Alkistis Pourtsidou, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [ ]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

import camb
from camb import model, initialpower

In [ ]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [ ]:
pi=np.pi

In [ ]:
#plots settings
font = {'size'   : 16, 'family':'STIXGeneral'}
axislabelfontsize='x-large'
plt.rc('font', **font)
plt.rcParams['legend.fontsize']='medium'

In [ ]:
#Fiducial cosmological parameters
c=3e5
hubble=0.678
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9
nss = 0.968

gamma=0.545

In [ ]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [ ]:
#calculate results for these parameters
results = camb.get_results(pars)
#print pars

In [ ]:
#Get matter power spectrum at z=0: P(k,z=0)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-6, maxkh=2.0, npoints = 200)

In [ ]:
#Construct P(k,z=0) interpolating function, in units of Mpc (no h)
Pkz0 = interp1d(kh*hubble, pk[0]/pow(hubble,3))

In [ ]:
ztest = 0.0

In [ ]:
#Spatially flat Universe

#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00
#Define the comoving distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)

In [ ]:
#Define the growth function in LCDM
def fg(zz):
    omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
    return pow(omz,gamma)

In [ ]:
#Get the growth factor 
def Dg_dz(zz):
    return -fg(zz)/(1+zz)

def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(ans)

In [ ]:
#fiducial bHI from Bull et al 2015
def bHI(zc):
    return 0.67+0.18*zc+0.05*pow(zc,2)

#fiducial OmHI Mario's fit
def OmHI(zc):
    return 0.00048+0.00039*zc-0.000065*pow(zc,2)

In [ ]:
#Construct matter P(k,z), no RSD
def Pkz(kk,zc):    
    return pow(Dgz(zc),2)*Pkz0(kk)

In [ ]:
#mean brightness temperature in [mK] Mario's fit from Bull et al 2015
def Tb(zc):
    return 0.0559+0.2324*zc-0.024*pow(zc,2)

In [ ]:
#Construct P_HI(k,z)
def PHI(kk,zc):
    return pow(Tb(zc),2)*pow(bHI(zc),2)*Pkz(kk,zc)

In [ ]:
kminn=1e-3
kmaxx=0.2

k1 = np.linspace(kminn,kmaxx,400)

plin = [PHI(k,ztest) for k in k1]

plt.loglog(k1,plin,color='blue')

plt.xlim(kminn,kmaxx)
#plt.ylim(1e2,5e5)

plt.xlabel("$k \, \mathrm{[Mpc^{-1}]}$")
plt.ylabel("$P_{\, \mathrm{HI}}(k) \, \mathrm{[mK^2 Mpc^3]}$")

plt.show()